In [1]:
# First off - load all python libraries we are going to need
import pandas as pd
import numpy as np
import os
#import plotly.plotly as py
#import plotly.graph_objs as go


from google.colab import auth
from google.cloud import bigquery
from google.colab import files

import matplotlib.pyplot as plt 
from mpl_toolkits import mplot3d
from matplotlib.pyplot import figure
import matplotlib.dates as mdates



from scipy.stats import ttest_ind

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import RidgeClassifier
!pip install regressors
from regressors import stats    


from sklearn.impute import SimpleImputer

from sklearn import metrics

import statsmodels.api as sm

# !pip install pygam
# from pygam import GAM, s, l , te ,LogisticGAM, LinearGAM

from patsy import dmatrices

import seaborn as sns
%matplotlib inline

# !pip install zepid
# import zepid as zp

#import plotly
#plotly.tools.set_credentials_file(username='harounchahed', api_key='ZHypTc119csyz5p0xWet')

  Created wheel for regressors: filename=regressors-0.0.3-py2.py3-none-any.whl size=12374 sha256=c42ffd26f1767ed83837825e5e630006786c2acc8931c51bcee4b2663f298c36
  Stored in directory: /root/.cache/pip/wheels/b2/89/fc/7867f77234d0033395f7ad9814f245b337139acaa06b085aa2
Successfully built regressors


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
auth.authenticate_user() #This will allow you to authenticate access to BigQuery4/rwDcMzFm8oGoxefjkisvhYNHOGbb11AmM1WbgDorhC0DBQNExxFg1yI

In [0]:
#This is a method that executes a desired SQL query on the database
project_id='ync-capstones'
os.environ["GOOGLE_CLOUD_PROJECT"]=project_id
# Read data from BigQuery into pandas dataframes.
def run_query(query):
  return pd.io.gbq.read_gbq(query, project_id=project_id, verbose=False, configuration={'query':{'useLegacySql': False}})

In [0]:
# Now load the data. In general you'd load the whole set of notes but that would take
# several minutes so for this example we're just going to use a subset
#notes = pd.read_csv('D.csv')
cohort = run_query('''
    select * 
    from `NMB.COHORT_BASELINE_P2`
    ''')

## Cohort

In [0]:
# cohort = cohort.set_index(['subject_id',	'hadm_id',	'icustay_id'], drop = True)

In [0]:
cohort.mv_starttime = pd.to_datetime(cohort.mv_starttime)
cohort.mv_endtime = pd.to_datetime(cohort.mv_endtime)
cohort.dod = pd.to_datetime(cohort.dod)

In [9]:
cohort.shape # number of patiens
# Sanity check: cohort.groupby(['subject_id', 'hadm_id','icustay_id']).size().count() 

(3820, 30)

In [0]:
# This code fixes few ages that are assigned an age >300 by mistake. We replace these ages with the average age of the dataset.
cohort.loc[cohort.age >= 300, 'age'] = 89

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
MSM_table_1H = "/content/drive/My Drive/Colab Notebooks/Capstone/MSM_table_1H.csv"
MSM_table_2H = "/content/drive/My Drive/Colab Notebooks/Capstone/MSM_table_2H.csv"
MSM_table_3H = "/content/drive/My Drive/Colab Notebooks/Capstone/MSM_table_3H.csv"
MSM_table_4H = "/content/drive/My Drive/Colab Notebooks/Capstone/MSM_table_4H.csv"
MSM_table_5H = "/content/drive/My Drive/Colab Notebooks/Capstone/MSM_table_5H.csv"

MSM_table_1H = pd.read_csv(MSM_table_1H)
MSM_table_2H = pd.read_csv(MSM_table_2H)
MSM_table_3H = pd.read_csv(MSM_table_3H)
MSM_table_4H = pd.read_csv(MSM_table_4H)
MSM_table_5H = pd.read_csv(MSM_table_5H)
# Dataset is now stored in a Pandas Dataframe

In [11]:
MSM_table_1H_trunc = truncate(MSM_table_1H, 0.03)
MSM_table_1H_trunc_location = "/content/drive/My Drive/Colab Notebooks/Capstone/MSM_table_1H_trunc.csv"
MSM_table_1H_trunc.to_csv(MSM_table_1H_trunc_location)

NameError: ignored

In [39]:
test = MSM_table_1H.groupby("subject_id")

In [0]:
test = test.mean()

In [47]:
test[test.tr_t != 0]

,Unnamed: 0,Unnamed: 0.1,hadm_id,icustay_id,mv_durations_hours_x,Spo2,Fio2,SF_ratio,NMBA_count,tr_t,tr_t_min_1,min_pao2fio2,max_peep,age,los,weight,height,BMI,female,oasis,elixhauser,explicit_sepsis,vaso,mortality,den,nom,IPTW,MV30_death,ICU_death,hos_death,ICU30_death,ICU90_death
subject_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
213.0,157866.0,157866.0,179315.0,253210.0,161.25000,97.593168,45.559007,2.210685,1.666667,0.012422,0.012422,140.00000,5.0,89.0,12.3049,82.0,NaN,NaN,0.0,35.0,8.0,1.0,1.0,1.0,0.184264,0.203436,0.868978,0.000000,0.000000,0.000000,0.000000,0.000000
250.0,312048.5,312048.5,124271.0,205951.0,241.00000,86.016949,62.733053,1.382853,0.736842,0.220339,0.216102,61.00000,12.0,23.0,10.6659,51.2,167.64,18.2,1.0,43.0,10.0,1.0,1.0,1.0,0.033048,0.032830,0.508760,0.004237,0.004237,0.004237,0.004237,0.004237
502.0,117135.5,117135.5,116367.0,209737.0,281.13333,96.184615,54.826923,1.795776,0.075581,0.026923,0.026923,95.00000,17.0,50.0,11.9479,84.7,178.00,26.7,0.0,52.0,9.0,1.0,1.0,1.0,0.022373,0.022498,0.498700,0.003846,0.003846,0.003846,0.003846,0.003846
592.0,372179.0,372179.0,180029.0,235336.0,172.00000,97.136364,53.846155,1.909969,1.652482,0.713287,0.713287,57.00000,20.0,63.0,7.3565,80.0,177.80,25.3,0.0,40.0,23.0,1.0,1.0,1.0,0.014521,0.014060,31.249248,0.006993,0.006993,0.006993,0.006993,0.006993
818.0,314088.0,314088.0,154890.0,249406.0,521.50000,96.558994,50.807544,1.963460,1.800000,0.050290,0.050290,70.00000,20.0,36.0,32.9795,72.0,170.18,24.9,1.0,39.0,0.0,0.0,1.0,0.0,0.023642,0.025503,0.185131,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97518.0,162562.5,162562.5,122851.0,213259.0,156.60000,95.256494,60.519481,1.635336,0.392405,0.097403,0.097403,76.25000,14.0,61.0,10.8085,90.0,175.00,29.4,1.0,35.0,13.0,0.0,1.0,0.0,0.025362,0.025731,0.169542,0.000000,0.000000,0.000000,0.000000,0.000000
98525.0,55431.5,55431.5,168505.0,294539.0,31.30000,96.444444,76.388889,1.272613,0.176471,0.111111,0.111111,72.00000,27.0,64.0,1.5074,148.0,180.34,45.5,0.0,47.0,13.0,1.0,1.0,1.0,0.215473,0.219904,0.818648,0.055556,0.055556,0.055556,0.055556,0.055556
98666.0,270594.5,270594.5,127829.0,273456.0,177.36667,99.162791,40.712209,2.451103,0.250000,0.017442,0.017442,112.00000,5.0,53.0,7.6034,53.0,165.10,19.4,1.0,44.0,6.0,0.0,1.0,1.0,0.011629,0.011610,0.765351,0.000000,0.000000,0.000000,0.000000,0.000000


In [48]:
test

,Unnamed: 0,Unnamed: 0.1,hadm_id,icustay_id,mv_durations_hours_x,Spo2,Fio2,SF_ratio,NMBA_count,tr_t,tr_t_min_1,min_pao2fio2,max_peep,age,los,weight,height,BMI,female,oasis,elixhauser,explicit_sepsis,vaso,mortality,den,nom,IPTW,MV30_death,ICU_death,hos_death,ICU30_death,ICU90_death
subject_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
9.0,360861.0,360861.0,150750.0,220597.0,118.33333,97.544248,52.212390,1.887159,NaN,0.0,0.0,125.00000,8.0,41.0,5.3231,104.00,182.88,31.1,0.0,35.0,12.0,0.0,1.0,1.0,0.711314,0.666058,0.924779,0.008850,0.008850,0.008850,0.008850,0.008850
32.0,310265.0,310265.0,175413.0,295037.0,12.75000,94.461538,50.000000,1.889231,NaN,0.0,0.0,147.00000,5.0,45.0,7.6854,120.00,182.88,35.9,0.0,29.0,-1.0,0.0,1.0,0.0,0.965423,0.942434,0.975952,0.000000,0.000000,0.000000,0.000000,0.000000
36.0,366175.5,366175.5,165660.0,241249.0,64.50000,93.835938,52.031250,1.811143,NaN,0.0,0.0,140.00000,10.0,72.0,4.3483,102.06,181.61,30.9,0.0,23.0,7.0,0.0,0.0,0.0,0.815287,0.940934,1.162842,0.000000,0.000000,0.000000,0.000000,0.000000
101.0,291213.5,291213.5,175533.0,233111.0,216.33333,98.811321,46.061321,2.175877,NaN,0.0,0.0,78.00000,5.0,82.0,9.8919,65.40,NaN,NaN,0.0,48.0,5.0,0.0,1.0,1.0,0.656044,0.647219,0.979413,0.004717,0.000000,0.004717,0.004717,0.004717
112.0,392482.5,392482.5,173177.0,251972.0,15.00000,99.500000,46.250001,2.200884,NaN,0.0,0.0,102.00000,5.0,89.0,1.1392,53.30,170.18,18.4,0.0,47.0,19.0,1.0,1.0,1.0,0.979685,0.985199,1.005625,0.125000,0.125000,0.125000,0.125000,0.125000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99873.0,377847.0,377847.0,143544.0,206270.0,13.61667,96.727273,51.363636,1.888953,NaN,0.0,0.0,86.00000,10.0,45.0,2.0048,103.40,188.00,29.3,0.0,29.0,0.0,0.0,1.0,0.0,0.966766,0.951139,0.983669,0.000000,0.000000,0.000000,0.000000,0.000000
99881.0,75422.0,75422.0,172327.0,214377.0,50.00000,95.906977,69.825581,1.465166,NaN,0.0,0.0,105.71429,15.0,77.0,2.1914,104.80,170.00,36.3,0.0,49.0,19.0,0.0,0.0,1.0,0.786119,0.965124,1.255081,0.023256,0.023256,0.023256,0.023256,0.023256
99912.0,96816.0,96816.0,189380.0,236183.0,257.93333,96.960784,44.421569,2.222893,NaN,0.0,0.0,115.00000,8.0,84.0,12.0867,84.00,175.00,27.4,0.0,43.0,5.0,1.0,1.0,1.0,0.590937,0.596790,0.991217,0.000000,0.000000,0.000000,0.000000,0.000000


## Structural Model

In [0]:
def truncate(MSM_table, p):
  MSM_table_trunc = MSM_table.copy()
  MSM_table_trunc.IPTW = np.where(MSM_table_trunc.IPTW > MSM_table_trunc.IPTW.quantile(1 - p), MSM_table_trunc.IPTW.quantile(1 - p), MSM_table_trunc.IPTW)
  MSM_table_trunc.IPTW = np.where(MSM_table_trunc.IPTW < MSM_table_trunc.IPTW.quantile(p), MSM_table_trunc.IPTW.quantile(p), MSM_table_trunc.IPTW)
  return MSM_table_trunc

In [0]:
def structural_model(MSM_table, p, interval):
  MSM_table_trunc = truncate(MSM_table, p)
  X = sm.add_constant(MSM_table_trunc[['tr_t_min_1', 'age', 'female', 	'oasis',	'elixhauser',	'explicit_sepsis',	'vaso']])
  y = MSM_table_trunc['MV30_death']

  # Instantiate a binomial family model with the logit function.
  MSM_model = sm.GLM(y, X, family = sm.families.Binomial(), links = sm.genmod.families.links.logit, freq_weights=MSM_table_trunc.IPTW.astype(np.float64))
  # MSM_model = sm.GLM(y, X, links = sm.genmod.families.links.logit, freq_weights = MSM_table.IPTW.astype(np.float64))

  MSM_result = MSM_model.fit()
  print("\n\nFor " + str(interval) + " and p = " + str(p) + " with IPTW description \n" + str(MSM_table_trunc.IPTW.describe()))
  print(MSM_result.summary())
  # print(MSM_result.summary().as_latex())
  return MSM_result

In [0]:
truncate_range = [0, 0.01, 0.02, 0.03, 0.04, 0.05]

In [40]:
structural_model(MSM_table_1H, 0.02, '1H')



For 1H and p = 0.02 with IPTW description 
count    394678.000000
mean          1.297176
std           1.739941
min           0.010298
25%           0.758501
50%           1.012464
75%           1.206259
max          11.412919
Name: IPTW, dtype: float64
                 Generalized Linear Model Regression Results                  
Dep. Variable:            ICU90_death   No. Observations:               394678
Model:                            GLM   Df Residuals:                511958.90
Model Family:                Binomial   Df Model:                            7
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -10244.
Date:                Tue, 07 Apr 2020   Deviance:                       20488.
Time:                        13:07:24   Pearson chi2:                 5.13e+05
No. Iterations:                     9                                         
Covariance Type:            nonro

In [0]:
for p in truncate_range: 
  structural_model(MSM_table_1H, p, '1H')

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:2542: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/statsmodels/genmod/families/family.py:890: RuntimeWarning: divide by zero encountered in true_divide
  n_endog_mu = self._clean((1. - endog) / (1. - mu))
/usr/local/lib/python3.6/dist-packages/statsmodels/genmod/families/family.py:890: RuntimeWarning: invalid value encountered in true_divide
  n_endog_mu = self._clean((1. - endog) / (1. - mu))




For 1H and p = 0 with IPTW description 
count    3.946780e+05
mean     7.130697e+12
std      2.399133e+14
min      1.623495e-10
25%      7.585010e-01
50%      1.012464e+00
75%      1.206259e+00
max      2.517809e+16
Name: IPTW, dtype: float64
                  Generalized Linear Model Regression Results                  
Dep. Variable:             MV30_death   No. Observations:                394678
Model:                            GLM   Df Residuals:       2814329131555663360
Model Family:                Binomial   Df Model:                             7
Link Function:                  logit   Scale:                           1.0000
Method:                          IRLS   Log-Likelihood:                     nan
Date:                Sun, 29 Mar 2020   Deviance:                           nan
Time:                        07:25:36   Pearson chi2:                  1.08e+19
No. Iterations:                   100                                          
Covariance Type:            nonrobu

/usr/local/lib/python3.6/dist-packages/statsmodels/genmod/families/family.py:942: RuntimeWarning: divide by zero encountered in true_divide
  special.gammaln(n - y + 1) + y * np.log(mu / (1 - mu)) +
/usr/local/lib/python3.6/dist-packages/statsmodels/genmod/families/family.py:942: RuntimeWarning: invalid value encountered in multiply
  special.gammaln(n - y + 1) + y * np.log(mu / (1 - mu)) +
/usr/local/lib/python3.6/dist-packages/statsmodels/genmod/families/family.py:943: RuntimeWarning: divide by zero encountered in log
  n * np.log(1 - mu)) * var_weights
/usr/local/lib/python3.6/dist-packages/statsmodels/genmod/families/family.py:943: RuntimeWarning: invalid value encountered in add
  n * np.log(1 - mu)) * var_weights




For 1H and p = 0.01 with IPTW description 
count    394678.000000
mean         13.670007
std         112.766421
min           0.001290
25%           0.758501
50%           1.012464
75%           1.206259
max        1095.934158
Name: IPTW, dtype: float64
                 Generalized Linear Model Regression Results                  
Dep. Variable:             MV30_death   No. Observations:               394678
Model:                            GLM   Df Residuals:               5395243.02
Model Family:                Binomial   Df Model:                            7
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -41881.
Date:                Sun, 29 Mar 2020   Deviance:                       83761.
Time:                        07:25:38   Pearson chi2:                 4.34e+06
No. Iterations:                    11                                         
Covariance Type:            nonro

In [0]:
for p in truncate_range: 
  structural_model(MSM_table_2H, p, '2H')

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:2542: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)




For 2H and p = 0 with IPTW description 
count    1.992030e+05
mean     1.378500e+07
std      4.055808e+08
min      8.459357e-07
25%      8.185017e-01
50%      1.011447e+00
75%      1.152654e+00
max      3.570349e+10
Name: IPTW, dtype: float64
                 Generalized Linear Model Regression Results                  
Dep. Variable:             MV30_death   No. Observations:               199203
Model:                            GLM   Df Residuals:         2746012589812.75
Model Family:                Binomial   Df Model:                            7
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -3.9035e+07
Date:                Sun, 29 Mar 2020   Deviance:                   7.8070e+07
Time:                        07:26:19   Pearson chi2:                 2.54e+12
No. Iterations:                    25                                         
Covariance Type:            nonrobust       

In [0]:
for p in truncate_range: 
  structural_model(MSM_table_3H, p, '3H')

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:2542: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)




For 3H and p = 0 with IPTW description 
count    1.341010e+05
mean     1.421727e+04
std      3.747626e+05
min      1.858435e-05
25%      8.502004e-01
50%      1.009643e+00
75%      1.117720e+00
max      2.920287e+07
Name: IPTW, dtype: float64
                 Generalized Linear Model Regression Results                  
Dep. Variable:             MV30_death   No. Observations:               134101
Model:                            GLM   Df Residuals:            1906550302.80
Model Family:                Binomial   Df Model:                            7
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -8.8619e+05
Date:                Sun, 29 Mar 2020   Deviance:                   1.7724e+06
Time:                        07:39:13   Pearson chi2:                 7.37e+08
No. Iterations:                    19                                         
Covariance Type:            nonrobust       

In [0]:
for p in truncate_range: 
  structural_model(MSM_table_4H, p, '4H')

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:2542: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)




For 4H and p = 0 with IPTW description 
count    101404.000000
mean        252.992787
std        6281.744054
min           0.000496
25%           0.873674
50%           1.008621
75%           1.098894
max      435731.587179
Name: IPTW, dtype: float64
                 Generalized Linear Model Regression Results                  
Dep. Variable:             MV30_death   No. Observations:               101404
Model:                            GLM   Df Residuals:              25654472.54
Model Family:                Binomial   Df Model:                            7
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -83909.
Date:                Sun, 29 Mar 2020   Deviance:                   1.6782e+05
Time:                        07:39:28   Pearson chi2:                 2.05e+07
No. Iterations:                    14                                         
Covariance Type:            nonrobus

In [0]:
for p in truncate_range: 
  structural_model(MSM_table_5H, p, '5H')

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:2542: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)




For 5H and p = 0 with IPTW description 
count    73965.000000
mean         1.296381
std          8.149260
min          0.000922
25%          0.962165
50%          1.006064
75%          1.044567
max       1115.592431
Name: IPTW, dtype: float64
                 Generalized Linear Model Regression Results                  
Dep. Variable:             MV30_death   No. Observations:                73965
Model:                            GLM   Df Residuals:                 95878.80
Model Family:                Binomial   Df Model:                            7
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -5117.4
Date:                Sun, 29 Mar 2020   Deviance:                       10235.
Time:                        07:26:03   Pearson chi2:                 9.37e+04
No. Iterations:                     8                                         
Covariance Type:            nonrobust       

In [0]:
# plt.rcParams['text.usetex']=True
# plt.rcParams['text.latex.unicode']=True
# sns.set(font_scale=1.5, rc={'text.usetex' : True})
import seaborn as sns
sns.set(style="whitegrid")
ax = sns.violinplot(x=np.log(MSM_table_1H.IPTW))
ax = sns.boxplot(x=np.log(MSM_table_1H.IPTW), showfliers=False, showbox=False, whis=[3,97], color = 'blue')
ax.set(xlabel='') #, ylabel='common ylabel')
ax.figure.savefig("IPTW_vPlot.pdf")
plt.show()

In [0]:
MSM_table_1H.IPTW.describe()

In [0]:
MSM_table_1H_trunc = truncate(MSM_table_1H , 0.03)
MSM_table_1H_trunc.IPTW.describe()

In [0]:
ax = sns.violinplot(x=MSM_table_1H_trunc.IPTW)